In [1]:
# cleaned using `utils/checknames.py`
from pipeline.train.det2.trainer import Det2Trainer
from config import TrainConfig, DataConfig, ModelConfig, DataPreparationConfig
import os, json, io, base64, cv2, glob
from pprint import pprint
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import labelme
# # configure trainer
# trainer = Det2Trainer(
#   data=DataConfig.POC102Assets_JsonDataset,
#   model=ModelConfig.POC102Assets_JsonModel,
#   cfg=TrainConfig)

# configure trainer
trainer = Det2Trainer(
  data=DataConfig.AllowedClassesDataset,
  model=ModelConfig.Allowed_ClassesModel,
  cfg=TrainConfig)
THING_CLASSES = DataConfig.AllowedClassesDataset.thing_classes

# removed the classes that exist in holdout / test but not in train. Need to implement sampler!

2021-05-13 16:53:40.161 | DEBUG    | pipeline.utils.file.save:dump:6 - saved ['store/model/Allowed_Classes/dta_cfg.bin']
2021-05-13 16:53:40.162 | DEBUG    | pipeline.utils.file.save:dump:6 - saved ['store/model/Allowed_Classes/mdl_cfg.bin']
2021-05-13 16:53:40.163 | DEBUG    | pipeline.utils.file.save:dump:6 - saved ['store/model/Allowed_Classes/trn_cfg.bin']


learning param base_lr            : 0.00025
learning param max_iter           : 10000
learning param batch_size_per_im  : 256
learning param ims_per_batch      : 16
learning param num_workers        : 25


In [2]:
!pip install loguru
!pip install joblib
!pip install labelme

  Using cached loguru-0.5.3-py3-none-any.whl (57 kB)
  Using cached aiocontextvars-0.2.2-py2.py3-none-any.whl (4.9 kB)
  Using cached contextvars-2.4-py3-none-any.whl
  Using cached immutables-0.15-cp36-cp36m-manylinux1_x86_64.whl (100 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached labelme-4.5.7-py3-none-any.whl
  Using cached matplotlib-3.2.2-cp36-cp36m-manylinux1_x86_64.whl (12.4 MB)
  Using cached PyQt5-5.15.4-cp36.cp37.cp38.cp39-abi3-manylinux2014_x86_64.whl (8.3 MB)
  Using cached imgviz-1.2.6-py3-none-any.whl
  Using cached PyQt5_Qt5-5.15.2-py3-none-manylinux2014_x86_64.whl (59.9 MB)
  Using cached PyQt5_sip-12.8.1-cp36-cp36m-manylinux1_x86_64.whl (278 kB)
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.3.4
    Uninstalling matplotlib-3.3.4:
      Successfully uninstalled matplotlib-3.3.4


# Run Visualisations on ANY Directory

In [2]:
import os
import joblib
import numpy
import copy
from loguru import logger

def dump(obj, saveby='./temp.bin'):
    ret = joblib.dump(obj, saveby)
    logger.debug(f"saved {ret}")

def load(from_path):
    return joblib.load(from_path)
def load_bin(from_path):
    return joblib.load(from_path)

from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.utils.visualizer import ColorMode

metadata = MetadataCatalog.get("dataset_train")
def get_seg_dict(predictor, on_im, save_dir, fname, THING_CLASSES):
    

    im = cv2.imread(on_im)
    # format at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    outputs = predictor(im)
    
    # print(len(outputs.keys())) #--> 1
    dic = outputs['instances'].__dict__
    
    pred_class_list =list(dic['_fields']['pred_classes'].cpu().numpy())
    print (pred_class_list)
    del dic['_fields']['pred_classes']
    dic['_fields']['pred_classes'] = []
    for obj in pred_class_list:
        dic['_fields']['pred_classes'].append(THING_CLASSES[obj])
        
    dic['_fields']['pred_classes'] = numpy.array(dic['_fields']['pred_classes'])
    v = Visualizer(im[:, :, ::-1],
            metadata=metadata, 
            scale=0.5, 
            instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    #plt.imshow(out.get_image()[:, :, ::-1])
    #plt.show()
    #plt.close()
    
    ## To save image please un comment below line.
    dic['_fields']['pred_classes']
    img = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_RGB2BGR)
    cv2.imwrite(save_dir+'/'+fname, img)
    
    #print(dic['_image_size']) #--> (4608, 9216)
    
    # print(dic['_fields'].keys()) #--> ['pred_boxes', 'scores', 'pred_classes', 'pred_masks']
    
    # print(dic['_fields']['pred_boxes'].tensor.cpu().detach().numpy())
    


    dic = {
        'imgShape': dic['_image_size'],
        'predClasses': dic['_fields']['pred_classes'],
        'predBoxes': dic['_fields']['pred_boxes'].tensor.cpu().detach().numpy(),
        'boxScores': dic['_fields']['scores'].cpu().detach().numpy()
    }
    return dic



def get_im_seg_info(images_dir, predictor, save_dir, THING_CLASSES, ext='jpg'):
    
    #  vis_metadata = load_bin(from_path=vis_metadata)  # train metadata

    write_dir = save_dir + f"eval_{images_dir[:-1].split('/')[-1]}" + '/'
    logger.debug(f"saving segmented images in {write_dir}")
    os.makedirs(write_dir, exist_ok=True)

    # check..
    # logger.debug(vis_metadata.thing_classes)

    #list_of_dics = []
    for im_name in os.listdir(images_dir):
        if im_name.split('.')[-1].lower() == ext.lower():
            
            dic = get_seg_dict(
                predictor=predictor,
                on_im=images_dir+im_name,
                save_dir = save_dir,
                fname= im_name,
                THING_CLASSES=THING_CLASSES
            )
            
            #from pprint import pprint
            #pprint(dic_str)
            dic['imageName'] = im_name
            
            
            joblib.dump((copy.deepcopy(dic), THING_CLASSES), write_dir + f'{im_name}.segmentation_info.bin')
            
            #list_of_dics.append(dic)

    return None   


THING_CLASSES = DataConfig.AllowedClassesDataset.thing_classes

In [3]:
from pipeline.eval.det2.get_model import GetTrained

predictor, cfg = (
        GetTrained(ModelConfig.Allowed_ClassesModel.__name__.replace("Model", ""))
        .fetch(thresh=0.5, cfg=True))
get_im_seg_info("/home/itis/Desktop/flag1/src/store/data/Allowed_Classes/test/", predictor, "store/data/Allowed_Classes/prediction/", THING_CLASSES, ext='jpg')

[INFO   ] checkpoint:load:138 - [Checkpointer] Loading from store/model/Allowed_Classes/model_final.pth ...
2021-05-13 16:54:50.637 | DEBUG    | __main__:get_im_seg_info:77 - saving segmented images in store/data/Allowed_Classes/prediction/eval_test/
/home/itis/Desktop/flag1/detectron2/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


[0]
[0]
[0]
[]
[0]
[0]
[]
[0]
[]
[0, 0]
[0]
[]
[]
[0]
[0]
[0, 0]
[0]
[0]
[0, 0]
[0]
[0]
[0, 0]
[0]
[0, 0]
[0]
[0]
[0]
[0]
[0]
[0]
[0, 0, 0]
[0]
[0]
[0, 0]
[0, 0]
[0]
[0]
[0]
[0]
[0, 0, 0]


In [19]:
len(THING_CLASSES)

103